## Capstone: EUR/USD Forecast - Time Series
---

### Problem Statement

Build a model to predict the future price of EUR/USD currency pair using historical daily close price data along with exogenous variables such as economic and financial indicators.

### Target Audience

This model is targeted to Hedge Funds, Financial Institutions and Multinational Corporations with investments/businesses in both United States and Europe. It allows them to mitigate their risk from market volatility by hedging against their open positions/obligations.

### Section

- [Import Libraries and Load Data](#Import-Libraries-and-Load-Data)
- [Data Cleaning](#Data-Cleaning)

### Import Libraries and Load Data

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import datetime as dt
from datetime import timedelta
pd.set_option('display.max_rows', 2060)
pd.set_option('display.max_columns', 75)

In [2]:
# Load features with daily data set from Jan 01 1999 to 31 Dec 2019 
# eur_usd target varaible
eur_usd = pd.read_csv(r'../datasets/EURUSD.csv')
gbp_usd = pd.read_csv(r'../datasets/GBPUSD.csv')
usd_jpy = pd.read_csv(r'../datasets/USDJPY.csv')
usd_chf = pd.read_csv(r'../datasets/USDCHF.csv')
usd_index = pd.read_csv(r'../datasets/DollarIndex.csv')
wti_crude = pd.read_csv(r'../datasets/WTICrude.csv')                        
sp500 = pd.read_csv(r'../datasets/S&P500.csv')
dow_jones = pd.read_csv(r'../datasets/DowJones.csv')
nasdaq = pd.read_csv(r'../datasets/Nasdaq.csv')
euro_n100 = pd.read_csv(r'../datasets/EURO_N100.csv')
cac40 = pd.read_csv(r'../datasets/CAC40.csv')
dax = pd.read_csv(r'../datasets/DAX.csv')
gold_usd = pd.read_csv(r'../datasets/gold_usd.csv')
brent_usd = pd.read_csv(r'../datasets/BrentCrude.csv')
fed_rate = pd.read_csv(r'../datasets/FedFundRate.csv')
euro_libor = pd.read_csv(r'../datasets/EuroLibor.csv')

# Load features with monthly data set from Jan 1999 to Dec 2019
us_unemp = pd.read_csv(r'../datasets/UsUnemployment.csv')
eu_unemp = pd.read_csv(r'../datasets/EuroUnemployment.csv')
us_retail = pd.read_csv(r'../datasets/USRetailSales.csv')
eu_retail = pd.read_csv(r'../datasets/EURetailSales.csv')
us_indus_prod = pd.read_csv(r'../datasets/USIndusProd.csv')
eu_indus_prod = pd.read_csv(r'../datasets/EuIndusProd.csv')
us_core_cpi = pd.read_csv(r'../datasets/USCoreCPI.csv')
eu_core_cpi = pd.read_csv(r'../datasets/EuroCoreCPI.csv')
us_cpi = pd.read_csv(r'../datasets/USCPI.csv')
eu_cpi = pd.read_csv(r'../datasets/EUCPI.csv')

In [3]:
# Combine features and target variable into daily dataset
features = [eur_usd, gbp_usd, usd_jpy, usd_chf, usd_index, wti_crude, sp500, dow_jones, nasdaq, euro_n100, cac40, 
            dax, gold_usd, brent_usd, fed_rate, euro_libor]
daily_df = reduce(lambda left,right: pd.merge(left,right,on=[' Date'], how='outer'), features)

In [4]:
# Combine monthly features into dataset
features_2 = [us_unemp, eu_unemp, us_retail, eu_retail, us_indus_prod, eu_indus_prod, us_core_cpi, eu_core_cpi,
            us_cpi, eu_cpi]
monthly_df = reduce(lambda left,right: pd.merge(left,right,on=[' Date'], how='outer'), features_2)

In [5]:
# Display the daily datasets
daily_df.head()

,Date,EURUSD Open,EURUSD Low,EURUSD High,EURUSD Close,EURUSD Adj. Close,GBPUSD Open,GBPUSD Low,GBPUSD High,GBPUSD Close,GBPUSD Adj. Close,USDJPY Open,USDJPY Low,USDJPY High,USDJPY Close,USDJPY Adj. Close,USDCHF Open,USDCHF Low,USDCHF High,USDCHF Close,USDCHF Adj. Close,USDCHF SMA (200D),USDCHF SMA (50D),DXY Close,DXY Adj. Close,DXY vs SPY Correlation,DXY Beta (DXY:SPY),CL1 Open,CL1 Low,CL1 High,CL1 Close,CL1 Adj. Close,SPX Open,SPX Low,SPX High,SPX Close,SPX Adj. Close,...,CCMP_EC Low,CCMP_EC High,CCMP_EC Close,CCMP_EC Adj. Close,N100 Open,N100 Low,N100 High,N100 Close,N100 Adj. Close,N100 SMA (200D),N100 SMA (50D),CAC_EC Open,CAC_EC Low,CAC_EC High,CAC_EC Close,CAC_EC Adj. Close,DAX_EC Open,DAX_EC Low,DAX_EC High,DAX_EC Close,DAX_EC Adj. Close,DAX_EC SMA (200D),DAX_EC SMA (50D),XAUUSD Open,XAUUSD Low,XAUUSD High,XAUUSD Close,XAUUSD Adj. Close,CO1 Open,CO1 Low,CO1 High,CO1 Close,CO1 Adj. Close,FDTR Close,FDTR Adj. Close,EMUEVOLVINTRAT Close,EMUEVOLVINTRAT Adj. Close
0,12/31/2019,1.1199,1.1199,1.1239,1.1233,1.1233,1.3112,1.3104,1.3217,1.3210,1.3210,108.8805,108.4730,108.8855,108.5060,108.5060,0.9693,0.9647,0.9697,0.9653,0.9653,0.9928,0.9881,96.71,96.71,0.10,-0.02,61.41,60.39,61.70,60.76,60.76,"3,223.51","3,212.30","3,229.81","3,227.57","3,227.57",...,"8,912.77","8,975.35","8,972.60","8,972.60","1,144.37","1,142.41","1,147.76","1,144.39","1,144.39","1,079.25","1,124.05","5,970.59","5,958.25","5,987.22","5,978.06","5,978.06",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,515.14","1,514.80","1,524.62","1,517.90","1,517.90",66.60,65.65,67.03,66.23,66.23,NaN,NaN,-0.42,-0.42
1,12/30/2019,1.1178,1.1173,1.1220,1.1198,1.1198,1.3092,1.3072,1.3150,1.3103,1.3103,109.4505,108.7765,109.4835,108.8335,108.8335,0.9774,0.9675,0.9774,0.9693,0.9693,0.9929,0.9886,96.80,96.80,-0.14,-0.02,61.51,60.76,62.15,61.40,61.40,"3,225.64","3,216.13","3,242.68","3,220.04","3,220.04",...,"8,909.19","9,006.36","8,945.99","8,945.99","1,155.09","1,146.56","1,156.44","1,146.56","1,146.56","1,078.79","1,122.95","6,028.96","5,982.22","6,037.70","5,982.22","5,982.22","13,290.71","13,119.19","13,305.71","13,135.20","13,135.20","12,343.74","13,112.13","1,510.95","1,510.95","1,515.93","1,515.03","1,515.03",66.92,66.10,67.68,66.61,66.61,NaN,NaN,-0.42,-0.42
2,12/27/2019,1.1098,1.1096,1.1187,1.1176,1.1176,1.2993,1.2972,1.3117,1.3075,1.3075,109.5875,109.3915,109.6055,109.4260,109.4260,0.9809,0.9731,0.9817,0.9745,0.9745,0.9930,0.9889,97.43,97.43,-0.15,-0.02,61.53,61.13,61.87,61.49,61.49,"3,236.62","3,233.94","3,251.80","3,238.92","3,238.92",...,"8,987.32","9,052.00","9,006.62","9,006.62","1,156.17","1,154.50","1,160.82","1,156.61","1,156.61","1,078.30","1,121.93","6,039.95","6,027.72","6,065.00","6,037.39","6,037.39","13,350.68","13,303.89","13,379.90","13,321.40","13,321.40","12,336.00","13,102.83","1,511.08","1,507.73","1,514.46","1,510.25","1,510.25",66.76,66.30,67.13,66.85,66.85,NaN,NaN,-0.43,-0.43
3,12/26/2019,1.1091,1.1083,1.1109,1.1103,1.1103,1.2963,1.2961,1.3015,1.3004,1.3004,109.3845,109.3660,109.6820,109.6275,109.6275,0.9804,0.9796,0.9819,0.9815,0.9815,0.9931,0.9891,97.63,97.63,-0.14,-0.02,61.07,60.77,61.66,61.53,61.53,"3,231.00","3,227.27","3,237.37","3,236.62","3,236.62",...,"8,968.45","9,022.46","9,022.39","9,022.39",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,503.69","1,497.90","1,512.31","1,511.69","1,511.69",66.15,66.05,66.94,66.76,66.76,NaN,NaN,-0.43,-0.43
4,12/25/2019,1.1086,1.1074,1.1089,1.1084,1.1084,1.2945,1.2925,1.2962,1.2952,1.2952,109.3855,109.3325,109.4170,109.3425,109.3425,0.9804,0.9803,0.9820,0.9816,0.9816,0.9932,0.9892,97.69,97.69,NaN,NaN,60.57,60.57,61.31,61.05,61.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,499.22","1,495.91","1,499.22","1,495.91","1,495.91",65.85,65.84,66.39,66.14,66.14,NaN,NaN,-0.43,-0.43


In [6]:
# Display the monthly datasets
monthly_df.head()

,Date,USURTOT Close,USURTOT Adj. Close,UMRTEMU Close,UMRTEMU Adj. Close,USARetailSalesYoY Close,USARetailSalesYoY Adj. Close,EMURetailSalesYoY Close,EMURetailSalesYoY Adj. Close,IP_YOY Close,IP_YOY Adj. Close,EUIPEMUY Close,EUIPEMUY Adj. Close,USACORECPIRATE Close,USACORECPIRATE Adj. Close,EUROAREACORINFRAT Close,EUROAREACORINFRAT Adj. Close,CPI_YOY Close,CPI_YOY Adj. Close,ECCPEMUY Close,ECCPEMUY Adj. Close
0,12/31/2019,3.5,3.5,7.4,7.4,5.8,5.8,1.3,1.3,-1.0,-1.0,-4.1,-4.1,2.3,2.3,1.3,1.3,2.3,2.3,1.3,1.3
1,11/30/2019,3.5,3.5,7.5,7.5,3.3,3.3,2.2,2.2,-0.8,-0.8,-1.5,-1.5,2.3,2.3,1.3,1.3,2.1,2.1,1.0,1.0
2,10/31/2019,3.6,3.6,7.5,7.5,3.1,3.1,1.4,1.4,-1.1,-1.1,-2.2,-2.2,2.3,2.3,1.1,1.1,1.8,1.8,0.7,0.7
3,9/30/2019,3.5,3.5,7.5,7.5,4.1,4.1,3.1,3.1,-0.1,-0.1,-1.7,-1.7,2.4,2.4,1.0,1.0,1.7,1.7,0.9,0.9
4,8/31/2019,3.7,3.7,7.4,7.4,4.1,4.1,2.1,2.1,0.4,0.4,-2.8,-2.8,2.4,2.4,0.9,0.9,1.7,1.7,1.0,1.0


### Data Cleaning

In [7]:
# Remove additional rows where target variable eur/usd is nan
# Create final df with the closing price columns for all features and date
daily_df = daily_df[daily_df['EURUSD Close'].notna()]
daily_df.columns = daily_df.columns.str.replace(' Close', '')
daily_final = daily_df[[' Date','EURUSD', 'GBPUSD', 'USDJPY', 'USDCHF', 'DXY', 'CL1', 'SPX', 'INDU_EC', 'CCMP_EC', 'N100', 
                    'CAC_EC', 'DAX_EC', 'XAUUSD', 'CO1', 'FDTR', 'EMUEVOLVINTRAT']]

In [8]:
# Create final df as well with the closing price columns for all monthly features and date
monthly_df.columns = monthly_df.columns.str.replace(' Close', '')
monthly_final = monthly_df[[' Date','USURTOT', 'UMRTEMU', 'USARetailSalesYoY', 'EMURetailSalesYoY', 'IP_YOY', 'EUIPEMUY',
                    'USACORECPIRATE', 'EUROAREACORINFRAT', 'CPI_YOY', 'ECCPEMUY']]

In [9]:
# Rename daily columns with shorter names and lowercase
new_columns = {
    ' Date': 'date',
    'EURUSD': 'eur/usd',
    'GBPUSD': 'gbp/usd',
    'USDJPY': 'usd/jpy',
    'USDCHF': 'usd/chf',
    'DXY': 'usd_index',
    'CL1': 'wti_crude',
    'SPX': 'snp_500',
    'INDU_EC': 'dow_jones',
    'CCMP_EC': 'nasdaq',
    'N100': 'euro_n100',
    'CAC_EC': 'cac_40',
    'DAX_EC': 'dax',
    'XAUUSD': 'gold_usd',
    'CO1': 'brent_crude',
    'FDTR': 'fed_rate',
    'EMUEVOLVINTRAT': 'euro_libor'
}

daily_final.rename(columns = new_columns, inplace=True)
daily_final.head()

C:\Users\Roshrav\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,date,eur/usd,gbp/usd,usd/jpy,usd/chf,usd_index,wti_crude,snp_500,dow_jones,nasdaq,euro_n100,cac_40,dax,gold_usd,brent_crude,fed_rate,euro_libor
0,12/31/2019,1.1233,1.3210,108.5060,0.9653,96.71,60.76,"3,227.57","28,489.40","8,972.60","1,144.39","5,978.06",NaN,"1,517.90",66.23,NaN,-0.42
1,12/30/2019,1.1198,1.3103,108.8335,0.9693,96.80,61.40,"3,220.04","28,438.99","8,945.99","1,146.56","5,982.22","13,135.20","1,515.03",66.61,NaN,-0.42
2,12/27/2019,1.1176,1.3075,109.4260,0.9745,97.43,61.49,"3,238.92","28,629.69","9,006.62","1,156.61","6,037.39","13,321.40","1,510.25",66.85,NaN,-0.43
3,12/26/2019,1.1103,1.3004,109.6275,0.9815,97.63,61.53,"3,236.62","28,577.38","9,022.39",NaN,NaN,NaN,"1,511.69",66.76,NaN,-0.43
4,12/25/2019,1.1084,1.2952,109.3425,0.9816,97.69,61.05,NaN,NaN,NaN,NaN,NaN,NaN,"1,495.91",66.14,NaN,-0.43


In [10]:
# Rename columns with shorter names and lowercase for monthly dataset
new_columns_2 = {
    ' Date': 'date',
    'USURTOT': 'us_unemp',
    'UMRTEMU': 'eu_unemp',
    'USARetailSalesYoY': 'us_retail',
    'EMURetailSalesYoY': 'eu_retail',
    'IP_YOY': 'us_indus',
    'EUIPEMUY': 'eu_indus',
    'USACORECPIRATE': 'us_core_cpi',
    'EUROAREACORINFRAT': 'eu_core_cpi',
    'CPI_YOY': 'us_cpi',
    'ECCPEMUY': 'eu_cpi',
}
monthly_final.rename(columns = new_columns_2, inplace=True)
monthly_final.head()

,date,us_unemp,eu_unemp,us_retail,eu_retail,us_indus,eu_indus,us_core_cpi,eu_core_cpi,us_cpi,eu_cpi
0,12/31/2019,3.5,7.4,5.8,1.3,-1.0,-4.1,2.3,1.3,2.3,1.3
1,11/30/2019,3.5,7.5,3.3,2.2,-0.8,-1.5,2.3,1.3,2.1,1.0
2,10/31/2019,3.6,7.5,3.1,1.4,-1.1,-2.2,2.3,1.1,1.8,0.7
3,9/30/2019,3.5,7.5,4.1,3.1,-0.1,-1.7,2.4,1.0,1.7,0.9
4,8/31/2019,3.7,7.4,4.1,2.1,0.4,-2.8,2.4,0.9,1.7,1.0


In [11]:
# Check the column data types and shape of daily dataset
daily_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5473 entries, 0 to 5472
Data columns (total 17 columns):
date           5473 non-null object
eur/usd        5473 non-null float64
gbp/usd        5473 non-null float64
usd/jpy        5473 non-null float64
usd/chf        5473 non-null float64
usd_index      5473 non-null float64
wti_crude      5284 non-null float64
snp_500        5283 non-null object
dow_jones      5384 non-null object
nasdaq         5292 non-null object
euro_n100      5120 non-null object
cac_40         5355 non-null object
dax            5339 non-null object
gold_usd       5469 non-null object
brent_crude    5463 non-null float64
fed_rate       285 non-null float64
euro_libor     5470 non-null float64
dtypes: float64(9), object(8)
memory usage: 769.6+ KB


In [12]:
# Check the column data types and shape of monthly dataset
# No null values for monthly data set
monthly_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 0 to 251
Data columns (total 11 columns):
date           252 non-null object
us_unemp       252 non-null float64
eu_unemp       252 non-null float64
us_retail      252 non-null float64
eu_retail      252 non-null float64
us_indus       252 non-null float64
eu_indus       252 non-null float64
us_core_cpi    252 non-null float64
eu_core_cpi    252 non-null float64
us_cpi         252 non-null float64
eu_cpi         252 non-null float64
dtypes: float64(10), object(1)
memory usage: 23.6+ KB


In [13]:
# Check for null values in daily data set
null_values = daily_final.isnull().sum().sort_values(ascending=False)
print(null_values[null_values > 0])

fed_rate       5188
euro_n100       353
snp_500         190
wti_crude       189
nasdaq          181
dax             134
cac_40          118
dow_jones        89
brent_crude      10
gold_usd          4
euro_libor        3
dtype: int64


In [14]:
# Euro N100 only started trading on 31st Dec 1999, used 31st value to bfill for the year(260 null values)
# Remaining null values due to public holiday, insert ffill values from previous business day
daily_final['euro_n100'] = daily_final['euro_n100'].ffill().bfill()
# CAC 40 missing values due to public holidays in France, use ffill values from previous business day, bfill for 1st Jan 1999
daily_final['cac_40'] = daily_final['cac_40'].ffill().bfill()
# DAX missing values due to public holidays in Germany, use ffill values from previous business day, bfill for 1st Jan 1999
daily_final['dax'] = daily_final['dax'].ffill().bfill()

# Missing values in dow_jones, sp_500 and nasdaq due to US public holidays
# Use ffill values from previous business day, bfill for 1st Jan 1999
daily_final['snp_500'] = daily_final['snp_500'].ffill().bfill()
daily_final['dow_jones'] = daily_final['dow_jones'].ffill().bfill()
daily_final['nasdaq'] = daily_final['nasdaq'].ffill().bfill()

# Missing values for commodities (wti, brent and gold) due to US public holidays
# Use ffill values from previous business day, bfill for 1st Jan 1999
daily_final['wti_crude'] = daily_final['wti_crude'].ffill().bfill()
daily_final['brent_crude'] = daily_final['brent_crude'].ffill().bfill()
daily_final['gold_usd'] = daily_final['gold_usd'].ffill().bfill()

# Missing values for rates (Fed Fund Rate and Euro Libor) due to public holidays and Fed Fund Rate is not available daily
# Use ffill values from previous business day, bfill for 1st Jan 1999
daily_final['fed_rate'] = daily_final['fed_rate'].ffill().bfill()
daily_final['euro_libor'] = daily_final['euro_libor'].ffill().bfill()

C:\Users\Roshrav\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Roshrav\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Roshrav\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [15]:
# Convert remaining daily columns with data type object to float except for date
col_to_float = ['snp_500', 'dow_jones', 'nasdaq',
             'euro_n100', 'cac_40', 'dax',
             'gold_usd']

for col in col_to_float:
    daily_final.loc[:,col] = daily_final.loc[:,col].apply(lambda x: x.replace(",", "")).astype(float)

C:\Users\Roshrav\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
# Convert date to date time data type in both daily and monthly data set
daily_final['date'] = pd.to_datetime(daily_final['date'])
monthly_final['date'] = pd.to_datetime(monthly_final['date'])
# Resample daily data set to monthly using their mean
daily_final_2 = daily_final.resample('M', on='date').mean()

C:\Users\Roshrav\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Roshrav\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# Merge resampled daily data with the monthly features and rename as final_df
# Set date as index and Save final_df to csv
final_df = pd.merge(daily_final_2, monthly_final, on='date')
final_df.set_index('date', inplace=True)
final_df.sort_index(inplace=True)
final_df.to_csv(r'..\datasets\final_df.csv', index='date')
daily_final.to_csv(r'..\datasets\final_df_2.csv', index='date')